# Sustainability for notebooks

This use case will showcase how existing notebooks projects provided by relevant European institutions can be assessed in terms of sustainability. It will use the collection of Jupyter Notebooks compiled by the [International GLAM Labs Community](https://www.glamlabs.io/computational-access-to-digital-collections).

This approach employs a workflow as shown in the following picture.

<img src="imgs/workflow.png" width="100%">

### Wikidata

Wikidata is a multilingual and collaborative edition platfrom providing access to a cross-domain repository, covering a wide diversity of topics. For example, the International GLAM Labs Community compiled a selection of Jupyter Notebook collection made available by relevant institutions. The metadata is openly available and can be retrieved using the [Wikidata SPARQL endpoint](https://w.wiki/HX6V):

```
SELECT ?s ?sLabel ?ownerLabel
WHERE {
  ?s wdt:P31 wd:Q7397 .
  ?s wdt:P361 wd:Q72936141 .
  ?s wdt:P127 ?owner
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }
}
```

First of all, we include all the python packages that are required to run the SPARQL query.

In [10]:
from SPARQLWrapper import SPARQLWrapper, JSON, RDFXML, POST

sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)

sparql.setMethod(POST)
sparql.setReturnFormat(JSON)

Now, we define the SPARQL query.

In [11]:
sparql.setQuery("""
SELECT ?s ?sLabel ?ownerLabel ?github ?website ?publicationdate (count(?dataset) as ?datasets)
WHERE {
  ?s wdt:P31 wd:Q7397 .
  ?s wdt:P361 wd:Q72936141 .
  ?s wdt:P127 ?owner .
  ?s wdt:P856 ?website .
  ?s wdt:P2283 ?dataset
  OPTIONAL {?s wdt:P577 ?publicationdate}
  OPTIONAL {?s wdt:P1324 ?github}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?s ?sLabel ?ownerLabel ?github ?website ?publicationdate
""")

Finally, we can retrieve the triples as JSON using the following code:

In [12]:
import json

try:
    ret = sparql.queryAndConvert()
    #print(ret)
    for r in ret["results"]["bindings"]:
        print(r["s"]["value"] + " - " + r["sLabel"]["value"] + " - " + r["ownerLabel"]["value"]+ " - " + r["datasets"]["value"])

    # we store the json in a file
    with open('data/glamlabs/metadata-glamlabs.json', 'w') as f:
        json.dump(ret, f)

except Exception as e:
    print(e)

http://www.wikidata.org/entity/Q111396450 - GLAM Jupyter Notebooks - Biblioteca Virtual Miguel de Cervantes - 11
http://www.wikidata.org/entity/Q111396660 - GLAM Workbench - Tim Sherratt - 15
http://www.wikidata.org/entity/Q111411199 - Data Foundry - Jupyter Notebooks - National Library of Scotland - 1
http://www.wikidata.org/entity/Q111421153 - Jupyter Notebooks at National Library of Estonia - National Library of Estonia - 1
http://www.wikidata.org/entity/Q111421205 - Jupyter Notebooks using the British Library’s Digital Collections and Data - British Library - 4
http://www.wikidata.org/entity/Q111450546 - Jupyter Notebooks LC for robots data exploration - Library of Congress - 1
http://www.wikidata.org/entity/Q123852539 - ONB Jupyter Notebooks - Austrian National Library - 1


### Integration with the ECCCH

[ECHOES](https://www.echoes-eccch.eu/faq/) is building a federated Knowledge Graph to allow for high level integration of resources. It will also serve as an entry point for all queries and requests related to any kind of information available within the Cultural heritage Cloud. The Knowledge Graph will use the proposed Heritage Digital Twin Ontology (HDTO) to unify descriptions and facilitate query and navigation. The current version of the ECHOES HDTO is available [here](https://www.echoes-eccch.eu/wp-content/uploads/2025/06/ECHOES_HDT_Ontology.pdf). The main vocabulary employed to describe the resources is [CIDOC-CRM](https://cidoc-crm.org/).

The following illustration shows how we modelled the outputs of this work in order to be integrated into the ECCCH.
<img width="80%" src="imgs/eccch-integration-steps.png">

And the following picture shows how we modelled the data using the vocabularies and ontologies. The class prov:Entity represents the code in the form of Jupyter Notebook which was generated by means of a prov:Activity, describing the work in this code, using a distribution of a dataset as an input (txt), and generating another distribution (ttl). The distributions are part of a dataset, which is also part of a catalog that was published by an organization.

<img width="80%" src="imgs/data-model-cidoc.png">

In [13]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import FOAF, RDF, RDFS, DCTERMS, VOID, DC, SKOS, OWL, XSD
import datetime

g = Graph()
g.bind("foaf", FOAF)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("void", VOID)
g.bind("skos", SKOS)
g.bind("owl", OWL)

schema = Namespace("https://schema.org/")
g.bind("schema", schema)

dcat = Namespace("http://www.w3.org/ns/dcat#")
g.bind("dcat", dcat)

wd = Namespace("http://www.wikidata.org/entity/")
g.bind("wd", wd)

cidoc_crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
g.bind("cidoc-crm", cidoc_crm)

prov = Namespace("http://www.w3.org/ns/prov#")
g.bind("prov", prov)

domain = 'https://example.org/'
domainLanguage = domain + 'map/'

##### We add the metadata of the dataset

In [14]:
# First, we create all the required URIS
glamlabs_catalog = URIRef(domain + "catalog/glamlabs")
glamlabs_org = URIRef(domain + "organization/glamlabs")
glamlabs_dataset = URIRef(domain + "dataset/glamlabs")
glamlabs_json = URIRef(domain + "distribution/glamlabs-json")
glamlabs_ttl = URIRef(domain + "distribution/glamlabs-ttl")

In [15]:
# We describe the dataset
g.add((glamlabs_catalog, RDF.type, schema.Dataset))
g.add((glamlabs_catalog, RDF.type, dcat.catalog))
g.add((glamlabs_catalog, RDFS.label, Literal("International GLAM Labs Community")))
g.add((glamlabs_catalog, schema.url, URIRef("https://www.glamlabs.io/computational-access-to-digital-collections")))
g.add((glamlabs_catalog, FOAF.homepage, URIRef("https://www.glamlabs.io/computational-access-to-digital-collections")))
g.add((glamlabs_catalog, schema.description, Literal("The information describing the Jupyter Notebook projects and the datasets reused has been introduced into Wikidata, a collaborative edition platform.")))
g.add((glamlabs_catalog, schema.name, Literal("The information describing the Jupyter Notebook projects and the datasets reused has been introduced into Wikidata, a collaborative edition platform.")))
g.add((glamlabs_catalog, DCTERMS.title, Literal("The information describing the Jupyter Notebook projects and the datasets reused has been introduced into Wikidata, a collaborative edition platform.")))
g.add((glamlabs_catalog, DCTERMS.publisher, URIRef(glamlabs_org))) # relation dataset-publisher
g.add((glamlabs_catalog, DC.title, Literal("International GLAM Labs Community")))
g.add((glamlabs_catalog, schema.license, URIRef('https://creativecommons.org/licenses/by/4.0/')))
g.add((glamlabs_catalog, dcat.dataset, glamlabs_dataset))

now = datetime.datetime.now()
g.add((glamlabs_catalog, schema.dateCreated, Literal(str(now)[:10])))

<Graph identifier=Ne0342b41736e42b482e54b314a648d29 (<class 'rdflib.graph.Graph'>)>

In [16]:
# We describe GLAM Labs
g.add((glamlabs_org, RDF.type, FOAF.Organization))
g.add((glamlabs_org, RDFS.label, Literal("International GLAM Labs Community")))
g.add((glamlabs_org, FOAF.homepage, URIRef("https://www.glamlabs.io/")))

<Graph identifier=Ne0342b41736e42b482e54b314a648d29 (<class 'rdflib.graph.Graph'>)>

In [17]:
# We describe the dataset
g.add((glamlabs_dataset, RDF.type, dcat.Dataset))
g.add((glamlabs_dataset, DCTERMS.title, Literal("Jupyter Notebooks projects compiled by the International GLAM Labs Community", lang="en")))
g.add((glamlabs_dataset, dcat.keyword, Literal("Collections as data")))
g.add((glamlabs_dataset, dcat.keyword, Literal("GLAM")))
g.add((glamlabs_dataset, dcat.keyword, Literal("Data")))
g.add((glamlabs_dataset, DCTERMS.issued, Literal(str(now)[:10])))
g.add((glamlabs_dataset, DCTERMS.language, URIRef("http://id.loc.gov/vocabulary/iso639-1/en")))
g.add((glamlabs_dataset, dcat.distribution, URIRef(glamlabs_json)))
g.add((glamlabs_dataset, dcat.distribution, URIRef(glamlabs_ttl)))

<Graph identifier=Ne0342b41736e42b482e54b314a648d29 (<class 'rdflib.graph.Graph'>)>

In [18]:
# We describe the distributions JSON and TTL 
g.add((glamlabs_json, RDF.type, dcat.Distribution))
g.add((glamlabs_json, dcat.downloadURL , URIRef("https://raw.githubusercontent.com/hibernator11/eccch-use-cases/refs/heads/main/data/glamlabs/metadata-glamlabs.json")))
g.add((glamlabs_json, DCTERMS.title, Literal("JSON distribution of Wikidata results", lang="en")))
g.add((glamlabs_json, DCTERMS.title, Literal("Distribución en JSON del conjunto de datos de la búsqueda de Wikidata", lang="es")))
g.add((glamlabs_json, dcat.mediaType, URIRef("http://www.iana.org/assignments/media-types/application/json")))
g.add((glamlabs_json, dcat.byteSize, Literal('1000000', datatype=XSD.integer)))

g.add((glamlabs_ttl, RDF.type, dcat.Distribution))
g.add((glamlabs_ttl, dcat.downloadURL , URIRef("https://raw.githubusercontent.com/hibernator11/eccch-use-cases/refs/heads/main/data/glamlabs/dataset_glamlabs.ttl")))
g.add((glamlabs_ttl, DCTERMS.title, Literal("TTL distribution of the Jupyter Notebooks results", lang="en")))
g.add((glamlabs_ttl, DCTERMS.title, Literal("Distribución en TTL del conjunto de datos de Jupyter Notebooks", lang="es")))
g.add((glamlabs_ttl, dcat.mediaType, URIRef("http://www.iana.org/assignments/media-types/application/n-triples")))
g.add((glamlabs_ttl, dcat.byteSize, Literal('360000', datatype=XSD.integer)))

<Graph identifier=Ne0342b41736e42b482e54b314a648d29 (<class 'rdflib.graph.Graph'>)>

### Now we link the notebooks and the distributions of the dataset created

In [19]:
sustainability_work = URIRef(domain + "sustainability-work/glamlabs")
notebooks = URIRef(domain + "notebooks/glamlabs")
g.add((notebooks, RDF.type, prov.Entity))
g.add((notebooks, DCTERMS.title, Literal("Sustainability for notebooks", lang="en")))
g.add((notebooks, prov.wasGeneratedBy, URIRef(sustainability_work)))
g.add((notebooks, dcat.mediatype, URIRef("https://www.iana.org/assignments/media-types/application/x-ipynb+json")))

now = datetime.datetime.now()
g.add((sustainability_work, RDF.type, prov.Activity))
g.add((sustainability_work, prov.startedAtTime, Literal(str(now)[:10])))
g.add((sustainability_work, prov.used, URIRef(glamlabs_json)))
g.add((sustainability_work, prov.generated, URIRef(glamlabs_ttl)))
g.add((sustainability_work, prov.endedAtTime, Literal(str(now)[:10])))

<Graph identifier=Ne0342b41736e42b482e54b314a648d29 (<class 'rdflib.graph.Graph'>)>

#### Read the JSON file 

In [28]:
import json

with open('data/glamlabs/metadata-glamlabs.json', newline='') as jsonfile:
    data = json.load(jsonfile)
    for item in data["results"]["bindings"]:
        print (item['s']['value'] + " - " + item['sLabel']['value'])

http://www.wikidata.org/entity/Q111396450 - GLAM Jupyter Notebooks
http://www.wikidata.org/entity/Q111396660 - GLAM Workbench
http://www.wikidata.org/entity/Q111411199 - Data Foundry - Jupyter Notebooks
http://www.wikidata.org/entity/Q111421153 - Jupyter Notebooks at National Library of Estonia
http://www.wikidata.org/entity/Q111421205 - Jupyter Notebooks using the British Library’s Digital Collections and Data
http://www.wikidata.org/entity/Q111450546 - Jupyter Notebooks LC for robots data exploration
http://www.wikidata.org/entity/Q123852539 - ONB Jupyter Notebooks


### Let's focus on the ONB example
In order to provide FAIR notebooks we would like to employ the [FAIR Jupyter initiative](https://fusion-jena.github.io/fairjupyter/). We will explore how to define a Jupyter Notebook using the classes and properties provided by this project.

The [ONB GitHub project](https://labs.onb.ac.at/gitlab/labs-team/austroflug-notebook) provides access to a Jupyter Notebook that we will describe according to the classes and properties provided by the ontologies [REPRODUCE-ME](https://sheeba-samuel.github.io/REPRODUCE-ME/doc/index-en.html), [PAV](https://pav-ontology.github.io/pav/pav.rdf), [DOAP](http://usefulinc.com/ns/doap#) and [P-PLAN](https://vocab.linkeddata.es/p-plan/index.html).

In [32]:
onb_uri = URIRef("http://www.wikidata.org/entity/Q123852539")
file_uri = URIRef(domain + "file/1")
github_uri = URIRef("https://labs.onb.ac.at/gitlab/labs-team/austroflug-notebook")

repr = Namespace("https://w3id.org/reproduceme")
g.bind("repr", repr)

pav = Namespace("https://pav-ontology.github.io/pav/pav.rdf")
g.bind("pav", pav)

doap = Namespace("http://usefulinc.com/ns/doap#")
g.bind("doap", doap)

p_plan = Namespace("http://purl.org/net/p-plan#")
g.bind("p-plan", p_plan)

g.add((onb_uri, RDF.type, repr.Notebook))
g.add((onb_uri, pav.retrievedFrom, github_uri))

g.add((github_uri, RDF.type, doap.GitRepository))

g.add((file_uri, RDF.type, repr.File))
g.add((file_uri, RDFS.label, Literal("austroflug-geopandas-visualize.ipynb")))
g.add((file_uri, pav.retrievedFrom, github_uri))

# TODO define cells and execution
# TODO define README file
# TODO define publications

<Graph identifier=Ne0342b41736e42b482e54b314a648d29 (<class 'rdflib.graph.Graph'>)>

### And we could the same with additional Jupyter Notebook repositories

In [33]:
# TODO

#### Store the data

In [34]:
g.serialize(destination="data/glamlabs/dataset_glamlabs.ttl")

<Graph identifier=Ne0342b41736e42b482e54b314a648d29 (<class 'rdflib.graph.Graph'>)>

#### Now we can query the graph using SPARQL

In [35]:
print('##### Properties:')

# Query the data in g using SPARQL
q = """
    SELECT distinct ?prop
    WHERE {
        ?s ?prop ?o .
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["prop"])

##### Properties:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/ns/dcat#byteSize
https://pav-ontology.github.io/pav/pav.rdfretrievedFrom
http://xmlns.com/foaf/0.1/homepage
http://purl.org/dc/terms/publisher
http://purl.org/dc/terms/issued
http://purl.org/dc/terms/language
http://www.w3.org/ns/prov#wasGeneratedBy
http://www.w3.org/ns/dcat#mediaType
http://www.w3.org/ns/dcat#keyword
http://www.w3.org/ns/dcat#downloadURL
http://www.w3.org/ns/prov#endedAtTime
http://www.w3.org/ns/dcat#dataset
https://schema.org/dateCreated
http://www.w3.org/ns/prov#used
http://purl.org/dc/terms/title
http://purl.org/dc/elements/1.1/title
https://schema.org/url
http://www.w3.org/2000/01/rdf-schema#label
https://schema.org/license
https://schema.org/name
http://www.w3.org/ns/dcat#distribution
http://www.w3.org/ns/dcat#mediatype
http://www.w3.org/ns/prov#startedAtTime
http://www.w3.org/ns/prov#generated
https://schema.org/description


As an example we can retrieve the metadata of our dataset

In [36]:
print('##### Dataset information:')

# Query the data in g using SPARQL
q = """
    PREFIX schema: <https://schema.org/>
    SELECT distinct ?p ?o
    WHERE {
        ?s a schema:Dataset .
        ?s ?p ?o
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["p"] + ": " + r["o"])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type:  does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: https://schema.org/Dataset does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: https://schema.org/Dataset does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type:  does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: http://www.w3.org/ns/dcat#catalog does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: http://www.w3.org/ns/dcat#catalog does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/2000/01/rdf-schema#label:  does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/2000/01/rdf-schema#label: Internatio

##### Dataset information:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: https://schema.org/Dataset
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: http://www.w3.org/ns/dcat#catalog
http://www.w3.org/2000/01/rdf-schema#label: International GLAM Labs Community
https://schema.org/url: https://www.glamlabs.io/computational-access-to-digital-collections
http://xmlns.com/foaf/0.1/homepage: https://www.glamlabs.io/computational-access-to-digital-collections
https://schema.org/description: The information describing the Jupyter Notebook projects and the datasets reused has been introduced into Wikidata, a collaborative edition platform.
https://schema.org/name: The information describing the Jupyter Notebook projects and the datasets reused has been introduced into Wikidata, a collaborative edition platform.
http://purl.org/dc/terms/title: The information describing the Jupyter Notebook projects and the datasets reused has been introduced into Wikidata, a collaborative edition platform.
h

#### Finally, we can use the ECCCH API to publish the data generated.

In [1]:
## To Be done!
# Integration with the ECCCH once the API and fina data model is available

### Publication & dissemination

This step involves the publication of the results obtained including the dataset and this notebook in different platforms such as the Social Sciences and Humanities Open Marketplace and Zenodo.


As an example, we will use the sandbox service of Zenodo. Note that if you want to use this code for production purposes, it is required to update the URL. First, we need to create an access token in this [link](https://zenodo.org/account/settings/applications/tokens/new/). Note that we also need a token for the sandbox Zenodo.

In [43]:
# https://developers.zenodo.org/
import requests
ACCESS_TOKEN = 'ChangeMe'

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}
r = requests.post('https://sandbox.zenodo.org/api/deposit/depositions',
                   json={},
                   headers=headers)
r.status_code
r.json()

{'created': '2026-01-12T17:41:07.632652+00:00',
 'modified': '2026-01-12T17:41:07.740296+00:00',
 'id': 424692,
 'conceptrecid': '424691',
 'metadata': {'access_right': 'open',
  'prereserve_doi': {'doi': '10.5281/zenodo.424692', 'recid': 424692}},
 'title': '',
 'links': {'self': 'https://sandbox.zenodo.org/api/deposit/depositions/424692',
  'html': 'https://sandbox.zenodo.org/deposit/424692',
  'badge': 'https://sandbox.zenodo.org/badge/doi/.svg',
  'files': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/files',
  'bucket': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c',
  'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/424692',
  'latest_draft_html': 'https://sandbox.zenodo.org/deposit/424692',
  'publish': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/actions/publish',
  'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/actions/edit',
  'discard': 'https://sandbox.zenodo.org/api/deposit/depos

Now, let’s upload a new file:

In [44]:
bucket_url = r.json()["links"]["bucket"]
deposition_id = r.json()["id"]

First, we create a zip file with the notebook and the requirements.txt file:

In [45]:
from zipfile import ZipFile

# List of files to include in the archive
file_list = ["Sustainability.ipynb", "requirements.txt"]

# Create ZIP file and write files into it
with ZipFile("output.zip", "w") as zipf:
   for file in file_list:
      zipf.write(file)

Then, we call the API:

In [47]:
filename = "output.zip"
path = "%s" % filename
headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}

''' 
The target URL is a combination of the bucket link with the desired filename
seperated by a slash.
'''
with open(path, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, filename),
        data=fp,
        headers=headers,
    )
r.json()

{'created': '2026-01-12T17:41:16.912746+00:00',
 'updated': '2026-01-12T17:41:17.052565+00:00',
 'version_id': 'f8cd6099-25aa-4105-aa41-91921a295466',
 'key': 'output.zip',
 'size': 28194,
 'mimetype': 'application/zip',
 'checksum': 'md5:4df29214004c115c3e7a0ab4a433da97',
 'is_head': True,
 'delete_marker': False,
 'links': {'self': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip',
  'version': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip?version_id=f8cd6099-25aa-4105-aa41-91921a295466',
  'uploads': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip?uploads=1'}}

We can also add metadata to the record:

In [49]:
data = {
     'metadata': {
         'title': 'Sustainability for notebooks',
         'upload_type': 'software',
         'description': 'This use case will showcase how existing notebooks projects provided by relevant European institutions can be assessed in terms of sustainability',
         'creators': [{'name': 'Candela, Gustavo',
                       'affiliation': 'University of Alicante'}]
     }
 }
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}
r = requests.put('https://sandbox.zenodo.org/api/deposit/depositions/%s' % deposition_id,
                  data=json.dumps(data),
                  headers=headers)
r.status_code

200

The last step is the publication:

In [50]:
headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}
r = requests.post('https://sandbox.zenodo.org/api/deposit/depositions/%s/actions/publish' % deposition_id,
                      headers=headers)
r.status_code
# 202

202

And now we can see the result in Zenodo:

<img src="imgs/zenodo-publication.png" width="70%">

We can reproduce the same with additional platforms such as [Wikidata](https://www.wikidata.org/) and the [Social Sciences and Humanities Open Marketplace](https://marketplace.sshopencloud.eu/about/api-documentation)

In the particular case of Wikidata, existing [python libraries](https://www.mediawiki.org/wiki/Manual:Pywikibot/Wikidata) can be used to extract and create entities.

### References

- Candela, G., Rosiński, C., & Margraf, A. (2025). A reproducible framework to publish and reuse Collections as data: the case of the European Literary Bibliography (Version 4, Vol. 965, Issue 170). Transformations: A DARIAH Journal . https://doi.org/10.46298/transformations.14729
- Gustavo Candela, Javier Pereda, Dolores Sáez, Pilar Escobar, Alexander Sánchez, Andrés Villa Torres, Albert A. Palacios, Kelly McDonough, and Patricia Murrieta-Flores. 2023. An Ontological Approach for Unlocking the Colonial Archive. J. Comput. Cult. Herit. 16, 4, Article 74 (December 2023), 18 pages. https://doi.org/10.1145/3594727
- https://developers.zenodo.org/#quickstart-upload
- https://www.echoes-eccch.eu/wp-content/uploads/2025/06/ECHOES_HDT_Ontology.pdf
- https://marketplace.sshopencloud.eu/about/api-documentation
- https://www.wikidata.org/
- https://cidoc-crm.org/sites/default/files/CRMdigv4.0.pdf
- https://www.w3.org/TR/prov-o/
- https://fusion-jena.github.io/fairjupyter/